<span>
<b>Authors:</b> 
<a href="http://------">Ornela Danushi </a>
<a href="http://------">Gerlando Gramaglia </a>
<a href="http://------">Domenico Profumo </a><br/>
<b>Python version:</b>  3.x<br/>
</span>

# Data Understanding & Preparation on Tennis Matches dataset 
Explore the dataset by studying the data quality, their distribution among several different features and the correlations.

The **central component** of the data science toolkit is **Pandas library** is a and it is used in conjunction with other libraries in that collection. Pandas is built on top of the **NumPy package**, meaning a lot of the structure of NumPy is used or replicated in Pandas. Data in pandas is often used to feed statistical analysis in **SciPy**, plotting functions from **Matplotlib**, and machine learning algorithms in Scikit-learn.

**Install Pandas**

Pandas is an easy package to install. Open up your terminal program (for Mac users) or command line (for PC users) and install it using either of the following commands: **conda install pandas OR pip install pandas.**

Alternatively, if you're using Jupyter notebook you can run a cell with: **!pip install pandas**

In [51]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

from collections import defaultdict
from scipy.stats.stats import pearsonr

## Loading the data set

Read the .csv file containing the data. The first line contains the list of attributes. The data is assigned to a Pandas dataframe.

In [52]:
df = pd.read_csv('tennis_matches.csv')#, sep=',', index_col=0) #alternative in case of json source df.to_json('filename.json')

In [53]:
print(df.head()) #print the first records of a df, 
# print()
# print(df.tail()) #print the last records of a df.

   Unnamed: 0 tourney_id tourney_name surface  draw_size tourney_level  \
0           0  2019-M020     Brisbane    Hard       32.0             A   
1           1  2019-M020     Brisbane    Hard       32.0             A   
2           2  2019-M020     Brisbane    Hard       32.0             A   
3           3  2019-M020     Brisbane    Hard       32.0             A   
4           4  2019-M020     Brisbane    Hard       32.0             A   

   tourney_date  match_num  winner_id winner_entry  ... l_2ndWon l_SvGms  \
0    20181231.0      300.0   105453.0          NaN  ...     20.0    14.0   
1    20181231.0      299.0   106421.0          NaN  ...      7.0    10.0   
2    20181231.0      298.0   105453.0          NaN  ...      6.0     8.0   
3    20181231.0      297.0   104542.0           PR  ...      9.0    11.0   
4    20181231.0      296.0   106421.0          NaN  ...     19.0    15.0   

   l_bpSaved l_bpFaced  winner_rank  winner_rank_points loser_rank  \
0       10.0      15.0      

In [54]:
#df.dtypes #return the type of each attribute but is already included in the df.info() called later

# Types of Attributes and basic checks 
## Data Quality with reference to Syntactic Accuracy

Check the data integrity, that is whether there are any empty cells or corrupted data. 
We will use for this purpose the Pandas function **info()**, which checks if there is any 
null value in any column. This function also checks data type for each column, as well as 
number of each data types and number of observations (rows).

Moreover we check if each attribute is syntactically correct according to the specifications

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186128 entries, 0 to 186127
Data columns (total 50 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          186128 non-null  int64  
 1   tourney_id          186073 non-null  object 
 2   tourney_name        186103 non-null  object 
 3   surface             185940 non-null  object 
 4   draw_size           186099 non-null  float64
 5   tourney_level       186099 non-null  object 
 6   tourney_date        186100 non-null  float64
 7   match_num           186101 non-null  float64
 8   winner_id           186073 non-null  float64
 9   winner_entry        25827 non-null   object 
 10  winner_name         186101 non-null  object 
 11  winner_hand         186082 non-null  object 
 12  winner_ht           49341 non-null   float64
 13  winner_ioc          186099 non-null  object 
 14  winner_age          183275 non-null  float64
 15  loser_id            186100 non-nul

### draw_size

number of players in the draw, often rounded up to the nearest power of 2. (For instance, a tournament with 28 players may be shown as 32.)

Analysis: since all powers of 2 are integers the idea is to convert to the 'int' format. In this case we can have undefined number errors so I consider undefined numbers 0 (controllare se è una soluzione corretta)

In [56]:
#print("Distinct Values in draw_size: \t", df.draw_size.unique())

dw = df['draw_size']
log= np.log2(df['draw_size'])
for i in range(0, len(log)):
    dec, inter = math.modf(log[i])
    if dec == 0:
        log[i] = inter
    else:
        log[i] = inter + 1
dw = pow(2, log)
df['draw_size'] = dw
df['draw_size'].unique()

array([ 32., 128.,  64.,   8.,   4.,  16.,   2.,  nan])

# Attributes category check

- Object Attributes:<br>
    - *tourney_id* - ***object*** 
    - *tourney_name* - ***object*** 
    - *surface* - ***object***
    - *tourney_level* - ***object***
    - *winner_entry* - ***object***
    - *winner_name* - ***object***
    - *winner_hand* - ***object***
    - *winner_ioc* - ***object***
    - *loser_entry* - ***object***
    - *loser_name* - ***object***
    - *loser_hand* - ***object***
    - *loser_ioc* - ***object*** 
    - *score* - ***object***
    - *round* - ***object***
    
    
- Numerical Attributes:<br>
    - *draw_size* - ***float64*** => ***int64*** done!
    - *tourney_date* - ***float64*** => ***Datetime64***
    - *match_num*- ***float64*** =>
    - *winner_id*- ***float64*** =>
    - *winner_ht*- ***float64*** => 
    - *winner_age*- ***float64*** => 
    - *loser_id*- ***float64*** =>
    - *loser_ht*- ***float64*** =>
    - *loser_age*- ***float64*** =>
    - *best_of* - ***float64***
    - *minutes* - ***float64***
    - *w_ace* - ***float64***
    - *w_df* - ***float64***
    - *w_svpt* - ***float64***
    - *w_1stIn* - ***float64***
    - *w_1stWon* - ***float64***
    - *w_2ndWon* - ***float64***
    - *w_SvGms* - ***float64***
    - *w_bpSaved* - ***float64***
    - *w_bpFaced* - ***float64***
    - *l_ace* - ***float64***
    - *l_df* - ***float64***
    - *l_svpt* - ***float64***
    - *l_1stIn* - ***float64***
    - *l_1stWon* - ***float64***
    - *l_2ndWon* - ***float64***
    - *l_SvGms* - ***float64***
    - *l_bpSaved* - ***float64***
    - *l_bpFaced* - ***float64***
    - *winner_rank* - ***float64***
    - *winner_rank_points* - ***float64***
    - *loser_rank* - ***float64***
    - *loser_rank_points* - ***float64***
    - *tourney_spectators* - ***float64*** 
    - *tourney_revenue* - ***float64***
    


In [48]:
print("Distinct Values in surface: \t", df.surface.unique())
print("Distinct Values in tourney_level: \t", df.tourney_level.unique())
print("Distinct Values in winner_entry: \t", df.winner_entry.unique())
print("Distinct Values in best_of: \t", df.best_of.unique())
print("Distinct Values in winner_hand: \t", df.winner_hand.unique())


Distinct Values in tourney_name: 	 ['Brisbane' 'Doha' 'Pune' ... 'W100 Nicholasville KY' 'W25 Las Vegas NV'
 nan]
Distinct Values in surface: 	 ['Hard' 'Clay' 'Grass' 'Carpet' nan]
Distinct Values in tourney_level: 	 ['A' 'P' 'G' 'I' 'M' 'PM' 'F' 'D' 'C' '15' '25' '60' '100' '80' '10' '50'
 '75' 'O' 'W' nan]
Distinct Values in winner_entry: 	 [nan 'PR' 'Q' 'WC' 'Alt' 'LL' 'SE' 'ALT' 'SR' 'JE' 'A' 'ITF' 'P' 'I' 'IR'
 'JR']
Distinct Values in best_of: 	 [ 3.  5. nan]
Distinct Values in winner_hand: 	 ['R' 'L' 'U' nan]


## winner and loser ht

check if the attributes contain negative values, we have some outliers and nan values

In [61]:
print(df['loser_ht'].describe())
print(df['winner_ht'].describe())
print(df['loser_ht'].unique())
print(df['winner_ht'].unique())

array([178., 198., 188., 183., 196.,  nan, 190., 193., 180., 174., 184.,
       182., 185., 177., 175., 170., 203., 208., 211., 168., 169., 172.,
       163., 179., 166., 173., 162., 176., 181., 191., 159., 155., 206.,
       164., 157., 171., 165., 167., 161., 189., 194.,   2., 160., 145.])

## Winner and loser ioc 

In [63]:
print(df['winner_ioc'].describe())
print(df['loser_ioc'].describe())
print(df['winner_ioc'].unique())
print(df['loser_ioc'].unique())

count     186099
unique       124
top          USA
freq       16464
Name: winner_ioc, dtype: object
count     186102
unique       154
top          USA
freq       16728
Name: loser_ioc, dtype: object
['JPN' 'RUS' 'FRA' 'AUS' 'CAN' 'BUL' 'GBR' 'SRB' 'USA' 'LAT' 'CZE' 'EST'
 'UKR' 'NED' 'CRO' 'BLR' 'CHI' 'SUI' 'POL' 'GER' 'LUX' 'ESP' 'ITA' 'GEO'
 'HUN' 'LTU' 'ARG' 'CYP' 'BIH' 'RSA' 'BEL' 'TUN' 'IND' 'BRA' 'AUT' 'POR'
 'NZL' 'URU' 'GRE' 'SVK' 'TPE' 'KAZ' 'PUR' 'KOR' 'ROU' 'MDA' 'SLO' 'CHN'
 'SWE' 'DEN' 'TUR' 'ESA' 'BAR' 'UZB' 'MNE' 'BOL' 'NOR' 'ECU' 'MEX' 'COL'
 'LIE' 'ISR' 'PAR' 'DOM' 'FIN' 'GUA' 'PER' 'INA' 'THA' 'PHI' 'EGY' 'ALG'
 'ZIM' 'PAK' 'MAR' 'HKG' 'IRL' 'LIB' 'SRI' 'VEN' 'MKD' 'PNG' 'SIN' 'GRN'
 'BAH' 'CUB' 'TRI' 'OMA' 'MLT' 'KGZ' 'MAS' 'BDI' 'MRI' 'SAM' 'KEN' 'ARM'
 'NAM' 'REU' 'UNK' 'MON' 'HAI' 'VIE' 'HON' 'PAN' 'CRC' 'SGP' 'TJK' 'POC'
 'IRI' 'PHL' 'MGL' 'GUM' 'GAB' 'NGR' 'GUD' 'CAM' 'CMR' 'KUW' 'MAD' 'DEU'
 'AND' 'NLD' 'NGA' 'GRC' nan]
['RUS' 'FRA' 'AUS' 'CAN' 'JPN' 'BUL' 'GBR

## winner and loser age
Cosa rappresenta la parte decimale??

In [68]:
print(df['winner_age'].describe())
print(df['loser_age'].describe())
print(df['winner_age'].unique())
print(df['loser_age'].unique())

count    183275.000000
mean         23.963517
std           4.462318
min          14.042437
25%          20.492813
50%          23.457906
75%          26.869268
max          95.000000
Name: winner_age, dtype: float64
count    179590.000000
mean         23.765932
std           4.629857
min          14.006845
25%          20.131417
50%          23.227926
75%          26.767967
max          74.485969
Name: loser_age, dtype: float64
[95.         22.88569473 29.00479124 ... 15.29089665 15.4880219
 33.51403149]
[22.88569473 33.70568104 31.88227242 ... 41.46201232 46.85284052
 14.14921287]


## minutes 
from this first analysis we found some outliers observing max value

In [73]:
print(df['minutes'].describe())

count    81660.000000
mean        97.675753
std         41.492701
min          0.000000
25%         72.000000
50%         91.000000
75%        119.000000
max       4756.000000
Name: minutes, dtype: float64


## w_ace l_ace

In [75]:
print(df['w_ace'].describe())
print(df['w_ace'].unique())
print(df['l_ace'].describe())
print(df['l_ace'].unique())


count    82310.000000
mean         4.813425
std          4.387105
min          0.000000
25%          2.000000
50%          4.000000
75%          7.000000
max         75.000000
Name: w_ace, dtype: float64
[ 3. 10.  2. 12.  5. 11.  1. 16. 21. 17. 15.  6.  7. 18. 44.  9.  8.  4.
  0. 14. 13. nan 33. 25. 20. 22. 26. 40. 19. 28. 29. 24. 30. 39. 23. 43.
 27. 32. 38. 35. 31. 48. 53. 36. 42. 34. 64. 37. 49. 45. 75. 41. 51. 61.
 46. 72. 52.]
count    82313.000000
mean         3.527875
std          3.828217
min          0.000000
25%          1.000000
50%          2.000000
75%          5.000000
max         67.000000
Name: l_ace, dtype: float64
[ 8. 17. 10.  1. 29. 12.  3.  5.  6. 13.  7.  4. 27. 22.  2.  0. 11. 20.
  9. nan 14. 36. 26. 15. 16. 47. 24. 18. 25. 21. 59. 28. 19. 23. 67. 38.
 37. 30. 34. 33. 31. 32. 40. 35. 52. 61. 53. 44. 45. 46. 56. 43. 39.]


## w_df l_df

In [76]:
print(df['w_df'].describe())
print(df['w_df'].unique())
print(df['l_df'].describe())
print(df['l_df'].unique())


count    82312.000000
mean         2.858174
std          2.421105
min          0.000000
25%          1.000000
50%          2.000000
75%          4.000000
max        114.000000
Name: w_df, dtype: float64
[  3.   1.   2.   8.   5.   0.   4.   6.   7.   9.  nan  10.  14.  12.
  11.  13.  18.  19.  15.  26.  17.  25.  16.  20.  21.  22.  23.  24.
 114.  72.  28.  45.]
count    82319.000000
mean         3.612556
std          2.608092
min          0.000000
25%          2.000000
50%          3.000000
75%          5.000000
max        114.000000
Name: l_df, dtype: float64
[  6.   2.   3.   5.   7.   0.   1.   4.  10.   8.   9.  nan  12.  11.
  13.  16.  15.  17.  14.  20.  21.  28.  18.  22.  19.  23.  26.  25.
  31. 114.  40.  36.]


## w_svpt l_svpt

In [79]:
print(df['w_svpt'].describe())
#print(df['w_svpt'].unique())
print(df['l_svpt'].describe())
#print(df['l_svpt'].unique())


count    82310.000000
mean        71.288069
std         25.524468
min          0.000000
25%         53.000000
50%         67.000000
75%         87.000000
max       1957.000000
Name: w_svpt, dtype: float64
count    82315.000000
mean        73.557335
std         24.985696
min          0.000000
25%         56.000000
50%         69.000000
75%         88.000000
max       1672.000000
Name: l_svpt, dtype: float64


## w_1stln l_1stln

In [83]:
print(df['w_1stIn'].describe())
#print(df['w_1stIn'].unique())
print(df['l_1stIn'].describe())
#print(df['l_1stIn'].unique())


count    82310.000000
mean        44.270477
std         16.951922
min          0.000000
25%         32.000000
50%         42.000000
75%         54.000000
max       1330.000000
Name: w_1stIn, dtype: float64
count    82304.000000
mean        44.557737
std         16.776201
min          0.000000
25%         33.000000
50%         42.000000
75%         54.000000
max        893.000000
Name: l_1stIn, dtype: float64


In [88]:
print(df['w_1stWon'].describe())
#print(df['w_1stIn'].unique())
print(df['l_1stWon'].describe())
#print(df['l_1stIn'].unique())


count    82312.000000
mean        32.130564
std         11.409554
min          0.000000
25%         24.000000
50%         30.000000
75%         38.000000
max        836.000000
Name: w_1stWon, dtype: float64
count    82311.000000
mean        28.028903
std         12.270939
min          0.000000
25%         19.000000
50%         26.000000
75%         35.000000
max        532.000000
Name: l_1stWon, dtype: float64


## TOURNEY_DATE

In [57]:
df['tourney_date'] = pd.to_datetime(df['tourney_date'], format='%Y%m%d')
df['tourney_date'] = [x.date() for x in df.tourney_date]
#df.tourney_date
#print("convert into Datatime64", df['tourney_date'])
#print("Distinct Values in tourney_date: \t", df.tourney_date.unique())



### tourney_id, tourney_date

tourney_id: has to be unique and the first four characters are always the year while the remaining part is random
tourney_date: eight digits, YYYYMMDD, usually the Monday of the tournament week.

Analysis:
we convert type of tourney_date in Datetime64

Split the value of the attribute in 2 parts, the first with 4 characters and the second with the remaing characters.
For both the parts build a set inserting each different year and id. 
Count how many wrong years are inserted through a NaN value. 
To detect the wrong ids is more difficult since they are random and don't follow a known a structure.
We can only conclude that the wrong id elements are those deriving from the wrong years.

NOTE: The decision to take is to delete or edit all these attributes.

In [30]:

#tourney_id=df['tourney_id'] 
#tourney_id.describe() 
tourney_year= df['tourney_id'].str[:4]
tourney_id= df['tourney_id'].str[5:]

tourney_year_set= set()
wrong_year= 0
#which_id=0
for i in tourney_year:    
    if math.isnan(float(i)):
        wrong_year += 1
        #print(tourney_id[which_id])
    else:
        tourney_year_set.add(i)
    #which_id += 1
print("Present years: "+ str(tourney_year_set)) #{'2018', '2019', '2016', '2020', '2021', '2017'}
print("Wrong years counting: "+ str(wrong_year)) #55

tourney_id_set=set()
for i in tourney_id:
    tourney_id_set.add(i)
#print(tourney_id_set)#come individuare i nan se i valori non sono per forza numerici? Questi valori nan sono parte di quelli individuati dall'anno nan

#print("\t"+"tourney_year" +"\t"+" tourney_id")
#print(+ tourney_year + "             " + tourney_id)

Present years: {'2018', '2016', '2020', '2019', '2021', '2017'}
Wrong years counting: 55


### tourney_name
is the name of the tourney

Analysis: with just viewing the values we see some NaN that have to be discovered. We build a set to view all the unique names.

In [9]:
print("Distinct Values in tourney_name: \t", df.tourney_name.unique())
tourney_name = df['tourney_name']
#tourney_name.describe() 
tourney_name_set = set()
for i in tourney_name:
    tourney_name_set.add(i)
#print(tourney_name_set)

### surface
kind of surface for the match

Analysis: Detected some unknown values

In [10]:
surface=df['surface']

surface_set= set()
wrong_surface= 0

for i in surface:
    if pd.isna(i):
        wrong_surface+=1
    else:
        surface_set.add(i)
        
print("surface_set: " + str(surface_set)) # {'Carpet', 'Hard', 'Clay', 'Grass'}
print("wrong_surface: " + str(wrong_surface)) # 188

surface_set: {'Carpet', 'Clay', 'Hard', 'Grass'}
wrong_surface: 188


### tourney_level
they are split for men and women.

○ For men: 'G' = Grand Slams, 'M' = Masters 1000s, 'A' = other tour-level
events, 'C' = Challengers, 'S' = Satellites/ITFs, 'F' = Tour finals and other
season-ending events, and 'D' = Davis Cup. F

○ For women, there are several additional tourney_level codes, including 'P' =
Premier, 'PM' = Premier Mandatory, and 'I' = International. The various levels
of ITFs are given by the prize money (in thousands), such as '15' = ITF
$15,000. Other codes, such as 'T1' for Tier I (and so on) are used for older
WTA tournament designations. 'D' is used for the Federation/Fed/Billie Jean
King Cup, and also for the Wightman Cup and Bonne Bell Cup.

○ There is also some competition which can be for both men and women: 'E' =
exhibition (events not sanctioned by the tour, though the definitions can be
ambiguous), 'J' = juniors, and 'T' = team tennis, which does yet appear
anywhere in the dataset but will at some point.


Analysis: Correct! nothing to do

In [11]:
tourney_level=df['tourney_level']

tourney_level_set= set()
wrong_tourneylevel= 0
men_levels=['G','M','A','C','S','F','D']
women_levels=['P','PM','I','ITF','WTA','D']
both_levels=['E','J','T']
#women_levels.append(men_levels)

gender=[] #['M','W','-']
loc=0

for i in tourney_level:
    if pd.isna(i):
        wrong_tourneylevel+=1
    else:
        if i in men_levels:
            gender.iloc[loc]='M'
        else if i in 
        tourney_level_set.add(i)
    loc +=1
    
print("tourney_level_set: " + str(tourney_level_set)) # {'M', 'W', 'PM', '100', 'D', 'O', '15', 'I', 'F', '60', 'G', '80', '50', 'A', 'P', '25', '75', 'C', '10'}
print("wrong_tourneylevel: " + str(wrong_tourneylevel)) # 29


SyntaxError: invalid syntax (<ipython-input-11-fa61e747b1a4>, line 19)

### winner_entry and loser_entry

'WC' = wild card, 'Q' = qualifier, 'LL' = lucky loser, 'PR' = protected
ranking, 'ITF' = ITF entry, and there are a few others that are occasionally used.

loser_entry: analogous

Analysis: Correct! nothing to do

In [ ]:
df['winner_entry'].head()

### winner_name and loser_name

name of player that win 

Analysis: Correct! nothing to do

In [ ]:
df['winner_name'].head()

### winner_hand and loser_hand
R= right, L = left, U = unknown. For ambidextrous players, this is their
serving hand.

Analysis: Correct! nothing to do

In [ ]:
df['winner_hand'].head()

### winner_ioc and loser_ioc

three-character country code

Analysis: Correct! nothing to do

In [ ]:
df['winner_ioc'].head()

### Score

Analysis: ?

In [ ]:
df['score'].head()

### Round

Analysis: Correct! nothing to do

In [ ]:
df['round'].head()

## Now let's start with analizyning each numerical Attributes


### tourney_date

eight digits, YYYYMMDD, usually the Monday of the tournament week.

Analysis: use pandas function for convert into date format

In [ ]:
df['tourney_date']=pd.to_datetime(df['tourney_date'], format='%Y%m%d')

print(df['tourney_date'])


### match_num

a match-specific identifier. Often starting from 1, sometimes counting down from 300, and sometimes arbitrary.

Analysis: I can convert in integer values and left nan in value 0 because my values is limited in a range [1,300]

In [ ]:
df['match_num'] = np.nan_to_num(df['match_num']).astype(int)
df['match_num'].head()

### winner_id and loser_id

the player_id used in this repo for the winner of the match.

In [ ]:
df['winner_id'].describe()

### winner_ht and loser_ht

height in centimetres, where available

In [ ]:
df['winner_ht']=np.nan_to_num(df['winner_ht']).astype(int)
df['winner_ht'].head()

### winner_age and loser_age

the age of the player, in years, depending on the date of the tournament

In [ ]:
df['winner_age'].head()

# Data Integration
We load another table containing the name and surname of any male and female person

In [ ]:
df_male = pd.read_csv('male_players.csv', sep=',') 
print(df_male.head())

print()

df_female = pd.read_csv('female_players.csv', sep=',') 
print(df_female.head())